In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('./911_calls_full.csv')
original_headers = list(df.columns.values)
print(original_headers)
print(len(df))


['Address', 'Type', 'Datetime', 'Latitude', 'Longitude', 'Report Location', 'Incident Number']
663810


/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# Get rid of rows which contains bad timestamp format
df['OverrideDateTime'] = df['Datetime'].apply(lambda c: (''.join(c.split()))[:-5] if len(c.split()) > 2 else None)
df = df[df.OverrideDateTime.notnull()]
print(len(df))
print(df.OverrideDateTime)

661764
0         11/09/201111:33:00PM
1         11/09/201111:32:00PM
2         11/09/201111:32:00PM
3         11/09/201111:29:00PM
4         11/09/201111:25:00PM
5         11/09/201111:17:00PM
6         11/09/201111:07:00PM
7         11/09/201111:05:00PM
8         11/09/201111:03:00PM
9         11/09/201111:02:00PM
10        11/09/201110:59:00PM
11        11/09/201110:56:00PM
12        11/09/201110:55:00PM
13        11/09/201110:55:00PM
14        11/09/201110:53:00PM
15        11/09/201110:52:00PM
16        11/09/201110:52:00PM
17        11/09/201110:46:00PM
18        11/09/201110:37:00PM
19        11/09/201110:29:00PM
20        11/09/201110:23:00PM
21        11/09/201110:23:00PM
22        11/09/201110:21:00PM
23        11/09/201110:11:00PM
24        11/09/201110:10:00PM
25        11/09/201110:04:00PM
26        11/09/201110:03:00PM
27        11/09/201110:02:00PM
28        11/09/201109:51:00PM
29        11/09/201109:47:00PM
                  ...         
663780    11/09/201706:41:00PM
6

In [3]:
df['hour'] = pd.to_datetime(df['OverrideDateTime'], format='%m/%d/%Y%I:%M:%S%p').dt.hour
print(df['hour'])

0         23
1         23
2         23
3         23
4         23
5         23
6         23
7         23
8         23
9         23
10        22
11        22
12        22
13        22
14        22
15        22
16        22
17        22
18        22
19        22
20        22
21        22
22        22
23        22
24        22
25        22
26        22
27        22
28        21
29        21
          ..
663780    18
663781    20
663782    21
663783    21
663784    21
663785    22
663786    22
663787    10
663788    15
663789    15
663790    17
663791    17
663792    17
663793    19
663794    20
663795    20
663796    23
663797    13
663798    17
663799    18
663800    13
663801    19
663802    23
663803    15
663804    19
663805    19
663806    19
663807    19
663808    19
663809    20
Name: hour, Length: 661764, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
# Split the dataset
train, test = train_test_split(df, test_size=0.2)
print(len(train))
print(len(test))

529411
132353


In [5]:
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
#('lr', clf1), ('rf', clf2), ('gnb', clf3)
classifier = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

# x is feature, y is label
train_x = train[['Latitude', 'Longitude', 'hour']]
train_x['Latitude'] = pd.to_numeric(train_x['Latitude'], errors='coerce') # transform the latitude attribtue
train_x['Longitude'] = pd.to_numeric(train_x['Longitude'],errors='coerce')

train_y = train[['Type']]

test_x = test[['Latitude', 'Longitude', 'hour']]
test_x['Latitude'] = pd.to_numeric(test_x['Latitude'],errors='coerce')
test_x['Longitude'] = pd.to_numeric(test_x['Longitude'],errors='coerce')

test_y = test[['Type']]

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [ ]:
le = preprocessing.LabelEncoder()
le.fit_transform(train_y.Type)

array([  8, 160,   8, ...,  93,   8, 160])

In [ ]:
train_x = train_x.fillna(train_x.mean()) # use mean to fillout missing value in train_x
model = classifier.fit(train_x, le.fit_transform(train_y))
train_score = model.score(train_x, le.fit_transform(train_y))
print(train_score) #Mean accuracy

/Users/yufan/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
test_x = test_x.fillna(test_x.mean())
le = preprocessing.LabelEncoder()
test_score = model.score(test_x, le.fit_transform(test_y))
print(test_score) 